In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Amazon ML + Sinergise Blog


## Data Acquisition

This notebook shows how to obtain raw data (Sentinel 2 satellite images) using Sentinel Hub and ground truth (polygons of tulip fields) from Geopedia. At the moment Geopedia contains only data from 2016. 2017 data and other crop types (i.e. other flower types) can be easily added.

###### Required WMS instance

The instance below was created by Sinergise for the purpuse of this blog. It was created on Jed's name using his email address (jed@amazon.de). Use this instance for accessing Sentinel 2 data via Sentinel Hub.

In [ ]:
WMS_INSTANCE = '71513b0b-264d-494a-b8c4-c3c36433db28'

In [ ]:
sentinel_hub_wms='https://services.sentinel-hub.com/ogc/wms/'+WMS_INSTANCE

In [ ]:
layers = {'tulip_field_2016':'ttl1904', 'tulip_field_2017':'ttl1905', 'arable_land_2017':'ttl1917'}

###### Required packages

For interactive display of Sentinel 2 images with tulip fields overlay a forked version of ipyleaflet is required, which can be found here: https://github.com/mocnik/ipyleaflet

To install it run:

pip install git+https://github.com/mocnik/ipyleaflet.git --upgrade

jupyter nbextension enable ipyleaflet --py --sys-prefix

In [ ]:
from ipyleaflet import Map, WmsLayer

## Interactive display

Create map and center it around Den Helder, Netherlands. You can zoom in/out and change the view using your mouse. 

In [ ]:
zoom_level = 14

In [ ]:
import math
earth_radius = 6372.7982e3
pixel_size = 2* math.pi * earth_radius * math.cos(math.radians(0))/2**(zoom_level+8)
print('Pixel dimension at zoom level %d equals %1.2f m.'%(zoom_level,pixel_size))

In [ ]:
m = Map(center=[52.9255665659715, 4.754333496093751], zoom=zoom_level, layout=dict(width='512px', height='512px')); m

#### Add Sentinel 2 layer to the map

In [ ]:
s2_layer = WmsLayer(url='https://services.sentinel-hub.com/v1/wms/'+WMS_INSTANCE, layers='TRUE_COLOR', tile_size=512)

In [ ]:
m.add_layer(s2_layer)

#### Add ground truth layer (all tulip fields from 2016)

In [ ]:
tulips = WmsLayer(url='http://service.geopedia.world/wms/ml_aws', layers=layers['tulip_field_2016'], tile_size=512, format='image/png', version='1.3.0', TRANSPARENT=True, opacity=0.4)

In [ ]:
m.add_layer(tulips)

In [ ]:
# m.remove_layer(tulips)

#### Define utilitiy function for converting points between two coordinate systems

In [ ]:
import pyproj

In [ ]:
def to_epsg3857(latlong_wgs84):
    epsg3857 = pyproj.Proj(init='epsg:3857')
    wgs84    = pyproj.Proj(init='EPSG:4326')
    return pyproj.transform(wgs84,epsg3857,latlong_wgs84[1],latlong_wgs84[0])

## Download patches

In [ ]:
bbox_3857 = [to_epsg3857(point) for point in m.bounds]; 

In [ ]:
import sys

Set path to directory containing DataRequest

In [ ]:
PATH = '../DataRequest'
sys.path.append(PATH)

In [ ]:
from DataRequest import S2Request, TulipFieldRequest

### Download Sentinel 2 data

#### RGB

In [ ]:
s2_request = S2Request(WMS_INSTANCE,layers='TRUE_COLOR', time=('2016-05-11'),bbox=bbox_3857,width=512,height=512,crs=3857)

In [ ]:
s2_rgb = s2_request.get_data()

In [ ]:
plt.imshow(s2_rgb[0])

#### All 13 bands

In [ ]:
s2_request = S2Request(WMS_INSTANCE,layers='ALL_BANDS', time=('2016-02-11'),bbox=bbox_3857,width=512,height=512,crs=3857)

In [ ]:
s2_allbands = s2_request.get_data()

In [ ]:
s2_allbands[0].shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(s2_allbands[0][:,:,1:4]*2.5)

In [ ]:
with open('test_512x512_13bands','wb') as fp:
    np.save(fp, s2_allbands[0])

#### Single band

In [ ]:
s2_bands = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B10','B11','B12']

In [ ]:
s2_request = S2Request(WMS_INSTANCE,layers=s2_bands[2], time=('2016-05-11'),bbox=bbox_3857,width=512,height=512,crs=3857)

In [ ]:
s2_b01 = s2_request.get_data()

In [ ]:
s2_b01[0].shape

### Download truth mask

In [ ]:
tulipFields = TulipFieldRequest(bbox=bbox_3857,width=512,height=512,crs=3857,layer=layers['tulip_field_2016'])

In [ ]:
tulip_field = tulipFields.get_data()

In [ ]:
plt.imshow(tulip_field[0])

## Download images for multiple acquisition times

In [ ]:
s2_request = S2Request(WMS_INSTANCE,layers='TRUE_COLOR', time=('2016-01-01','2016-05-11'),bbox=bbox_3857,width=512,height=512,crs=3857)

In [ ]:
s2_rgb_2016 = s2_request.get_data()

In [ ]:
print('Downloaded %d images in given time range:'%(len(s2_rgb_2016)))

In [ ]:
import urllib.parse as urlparse

dates = [urlparse.parse_qs(urlparse.urlparse(item.url).query)['time'] for item in s2_request.download_list]

In [ ]:
fig = plt.figure(figsize=(8,8), dpi=300)
for index in range(0,max(12,len(s2_rgb_2016))):
    plt.subplot(4,3, index+1)
    plt.axis('off')
    plt.imshow(s2_rgb_2016[index])
    plt.title(dates[index] , fontsize=4, y=1.0)

plt.show()